In [1]:
import argparse
import os
from util import *
import tensorflow as tf
# import keras as K
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras import layers

2022-09-25 21:12:21.935497: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-25 21:12:22.178128: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-09-25 21:12:22.780221: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-09-25 21:12:22.780299: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such 

In [2]:
x_train, y_train, x_test, y_test = load_tensorflow_data('./tinyml_contest_data_training/', './data_indices')

In [3]:
x_train.shape
x_train = tf.convert_to_tensor(x_train, dtype=tf.float32)
# y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)
x_test = tf.convert_to_tensor(x_test, dtype=tf.float32)
y_test = tf.convert_to_tensor(y_test, dtype=tf.float32)



2022-09-25 21:12:40.557245: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-25 21:12:43.688001: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 20343 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090 Ti, pci bus id: 0000:73:00.0, compute capability: 8.6


In [4]:
def fbeta_weighted(true, pred): #shapes (batch, 4)
    beta = 2
    #for metrics include these two lines, for loss, don't include them
    #these are meant to round 'pred' to exactly zeros and ones
    #predLabels = K.argmax(pred, axis=-1)
    #pred = K.one_hot(predLabels, 4) 

    ground_positives = K.sum(true, axis=0) + K.epsilon()       # = TP + FN
    pred_positives = K.sum(pred, axis=0) + K.epsilon()         # = TP + FP
    true_positives = K.sum(true * pred, axis=0) + K.epsilon()  # = TP
        #all with shape (4,)
    
    precision = true_positives / pred_positives 
    recall = true_positives / ground_positives
        #both = 1 if ground_positives == 0 or pred_positives == 0
        #shape (4,)

    # f_bata = 2 * (precision * recall) / (precision + recall + K.epsilon())
        #still with shape (4,)
    f_bata = (1 + beta**2) * ((precision * recall) / ((beta**2 * precision) + recall + K.epsilon()))

    weighted_fbeta = f_bata * ground_positives / K.sum(ground_positives) 
    weighted_fbeta = K.sum(weighted_fbeta)

    
    return 1 - weighted_fbeta #for metrics, return only 'weighted_fbeta'

In [5]:
def fbeta_loss(y_true, y_pred):
    beta = 2
    y_true = K.one_hot(y_true, 2)
    y_pred = K.softmax(y_pred, axis=1)
    # y_true = K.cast(y_true, 'float')
    
    tp = K.sum(y_true * y_pred, axis=0)
    tn = K.sum((1 - y_true) * (1 - y_pred), axis=0)
    fp = K.sum((1 - y_true) * y_pred, axis=0)
    fn = K.sum(y_true * (1 - y_pred), axis=0)

    precision = tp / (tp + fp + K.epsilon())
    recall = tp / (tp + fn + K.epsilon())
    # precision = tp / (tp + fp)
    # recall = tp / (tp + fn)

    f_bata = (1 + beta**2) * ((precision * recall) / ((beta**2 * precision) + recall + K.epsilon()))
    # f_bata = (1+beta**2) * (precision * recall) / ((beta**2)*precision + recall)
    return 1 - K.mean(f_bata)

In [6]:
from ctypes.wintypes import PINT


def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    # f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

def f1_loss(y_true, y_pred):
    print(y_true)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    # f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return 1 - K.mean(f1)

In [7]:
def vgg_model():
    model = tf.keras.Sequential()
    
    model.add(tf.keras.layers.Conv2D(filters=2,
                                     kernel_size=(3, 1),
                                     strides=(1, 1),
                                     padding='same',
                                     data_format = 'channels_first',
                                     activation=tf.keras.activations.relu,
                                     input_shape=(1, 1250, 1)))

    model.add(tf.keras.layers.MaxPool2D(pool_size=(1, 5),
                                        strides=(1, 5)))

    model.add(tf.keras.layers.Conv2D(filters=4,
                                     kernel_size=(3, 1),
                                     strides=(1, 1),
                                     padding='same',
                                     data_format = 'channels_first',
                                     activation=tf.keras.activations.relu))

    model.add(tf.keras.layers.MaxPool2D(pool_size=(1, 5),
                                        strides=(1, 5)))

    model.add(tf.keras.layers.Conv2D(filters=8,
                                     kernel_size=(3, 1),
                                     strides=(1, 1),
                                     padding='same',
                                     data_format = 'channels_first',
                                     activation=tf.keras.activations.relu))

    model.add(tf.keras.layers.Conv2D(filters=8,
                                     kernel_size=(3, 1),
                                     strides=1,
                                     padding='same',
                                     data_format = 'channels_first',
                                     activation=tf.keras.activations.relu))

    model.add(tf.keras.layers.MaxPool2D(pool_size=(1, 5),
                                        strides=(1, 5)))

    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Flatten())

    model.add(tf.keras.layers.Dense(units=9,
                                    activation=tf.keras.activations.relu))
    model.add(tf.keras.layers.Dense(units=2,
                                    activation=None))

    return model

In [8]:
model = vgg_model()

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 2, 1250, 1)        8         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 2, 250, 1)        0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 4, 250, 1)         28        
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 4, 50, 1)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 8, 50, 1)          104       
                                                                 
 conv2d_3 (Conv2D)           (None, 8, 50, 1)          2

In [10]:
# loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# loss_fn = fbeta_weighted
loss_fn = fbeta_loss

model.compile(optimizer='adam', loss=loss_fn, metrics=["acc"])
# model.compile(optimizer='adam', loss=f1_loss, metrics=['accuracy', f1])


In [11]:
# cross_entropy
model.fit(x_train, y_train, batch_size = 256, epochs=200)

Epoch 1/200


2022-09-25 21:12:47.481332: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8302
2022-09-25 21:12:49.103066: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-09-25 21:12:49.172892: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


97/97 [==============================] - 5s 8ms/step - loss: 0.6304 - acc: 0.6508
Epoch 2/200
97/97 [==============================] - 1s 8ms/step - loss: 0.4924 - acc: 0.7528
Epoch 3/200
97/97 [==============================] - 1s 7ms/step - loss: 0.3863 - acc: 0.8397
Epoch 4/200
97/97 [==============================] - 1s 9ms/step - loss: 0.3235 - acc: 0.8832
Epoch 5/200
97/97 [==============================] - 1s 7ms/step - loss: 0.2921 - acc: 0.8944
Epoch 6/200
97/97 [==============================] - 1s 8ms/step - loss: 0.2690 - acc: 0.8998
Epoch 7/200
97/97 [==============================] - 1s 9ms/step - loss: 0.2490 - acc: 0.9065
Epoch 8/200
97/97 [==============================] - 1s 7ms/step - loss: 0.2252 - acc: 0.9114
Epoch 9/200
97/97 [==============================] - 1s 8ms/step - loss: 0.2054 - acc: 0.9168
Epoch 10/200
97/97 [==============================] - 1s 9ms/step - loss: 0.1938 - acc: 0.9217
Epoch 11/200
97/97 [==============================] - 1s 8ms/step - los

In [12]:
model.evaluate(x_test,  y_test)

176/176 [==============================] - 1s 7ms/step - loss: 0.2637 - acc: 0.9339


[0.26374462246894836, 0.9338666796684265]

In [11]:
model.fit(x_train, y_train, batch_size = 256, epochs=200)

Epoch 1/200


2022-09-25 19:48:37.412051: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8302
2022-09-25 19:48:39.210457: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-09-25 19:48:39.277634: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


97/97 [==============================] - 6s 9ms/step - loss: 0.5005 - acc: 0.5409
Epoch 2/200
97/97 [==============================] - 1s 9ms/step - loss: 0.5003 - acc: 0.5329
Epoch 3/200
97/97 [==============================] - 1s 8ms/step - loss: 0.5003 - acc: 0.5249
Epoch 4/200
97/97 [==============================] - 1s 9ms/step - loss: 0.5003 - acc: 0.5242
Epoch 5/200
97/97 [==============================] - 1s 8ms/step - loss: 0.5003 - acc: 0.5205
Epoch 6/200
97/97 [==============================] - 1s 8ms/step - loss: 0.5003 - acc: 0.5190
Epoch 7/200
97/97 [==============================] - 1s 9ms/step - loss: 0.5003 - acc: 0.5190
Epoch 8/200
97/97 [==============================] - 1s 13ms/step - loss: 0.5003 - acc: 0.5190
Epoch 9/200
97/97 [==============================] - 1s 9ms/step - loss: 0.5003 - acc: 0.5187
Epoch 10/200
97/97 [==============================] - 1s 10ms/step - loss: 0.5003 - acc: 0.5190
Epoch 11/200
97/97 [==============================] - 1s 8ms/step - l

In [ ]:
model.evaluate(x_test,  y_test)

176/176 [==============================] - 1s 4ms/step - loss: -0.5624 - acc: 0.5753


[-0.5623986721038818, 0.5752888917922974]